# Chunking

Demo tutorial for how to use nlp_toolkit to train sequence labeling model and predict new samples. The task we choose is noun phrases labeling.

The dataset includes working experience texts from different cv, and we want to label noun phrases in given text.

Available models:

1. WordRNN
2. CharRNN
3. IDCNN

In [1]:
import sys
sys.path.append('../')
from nlp_toolkit import Dataset, Labeler

/opt/wangyilei/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Data Processing

### Load config dict

In [2]:
import yaml
config = yaml.load(open('../nlp_toolkit/config_sequence_labeling.yaml'))
config['data']['inner_char'] = True

### Load data

In [3]:
dataset = Dataset(fname='../data/cv_word.txt', task_type='sequence_labeling', mode='train', config=config)

2018-11-18 23:43:35,198 - data.py[line:130] - INFO: data loaded


In [4]:
for x, y in zip(dataset.texts[0:10], dataset.labels[0:10]):
    print(list(zip(x, y)))

[('主要', 'O'), ('帮助', 'O'), ('工地', 'B-Chunk'), ('师傅', 'E-Chunk'), ('一起', 'O'), ('超平', 'O'), (',', 'O'), ('防线', 'O'), ('工作', 'O')]
[('协助', 'O'), ('线', 'O'), ('上', 'O'), ('、', 'O'), ('线', 'B-Chunk'), ('下', 'I-Chunk'), ('活动', 'E-Chunk'), ('的', 'O'), ('执行', 'O')]
[('执行', 'O'), ('各项', 'O'), ('培训', 'O'), ('相关', 'O'), ('的', 'O'), ('各项', 'O'), ('工作', 'B-Chunk'), ('流程', 'E-Chunk')]
[('云南', 'O'), (':', 'O'), ('曲靖', 'O'), ('、', 'O'), ('昭通', 'O'), ('下属', 'O'), ('的', 'O'), ('5', 'O'), ('个', 'O'), ('县级', 'O'), ('供电', 'B-Chunk'), ('公司', 'E-Chunk'), ('10', 'O'), ('个', 'O'), ('供电所', 'O')]
[('担任', 'O'), ('培训', 'O'), ('学校', 'B-Chunk'), ('英语', 'I-Chunk'), ('讲师', 'E-Chunk'), ('一', 'O'), ('职', 'O'), ('和', 'O'), ('学生', 'B-Chunk'), ('管理', 'E-Chunk')]
[('搜寻', 'O'), ('招标', 'B-Chunk'), ('公告', 'E-Chunk'), (',', 'O'), ('告知', 'O'), ('领导', 'O'), ('及', 'O'), ('业务', 'B-Chunk'), ('人员', 'E-Chunk'), (',', 'O'), ('确认', 'O'), ('是否', 'O'), ('报名', 'O')]
[('2001', 'O'), ('/', 'O'), ('10', 'O'), ('--', 'O'), ('2002', 'O'), ('/'

### Transform data to index

In [5]:
# if we want to use pre_trained embeddings, we need a gensim-format embedding file
x, y, config = dataset.transform()
print(x['token'].shape, y.shape)

2018-11-18 23:43:47,123 - data.py[line:215] - INFO: texts and labels have been transformed to number index
2018-11-18 23:43:47,124 - data.py[line:223] - INFO: Use Embeddings from Straching


(57415, 80) (57415, 80, 4)


if your want to see the vocab and label index mapping dict

In [6]:
# dataset.transformer._word_vocab._token2id

In [7]:
# dataset.transformer._label_vocab._token2id

In [8]:
transformer = dataset.transformer

## Chunking Labeling

### Define Sequence Labeler

avialiable models: model_name_list = ['word_rnn', 'char_rnn', 'idcnn']

In [9]:
model_name='word_rnn'
seq_labeler = Labeler(model_name=model_name, transformer=transformer, seq_type='bucket', config=config)

### Train model

In [ ]:
trained_model = seq_labeler.train(x, y)

2018-11-18 23:43:50,095 - trainer.py[line:129] - INFO: word_rnn model structure...


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
char (InputLayer)               (None, None, None)   0                                            
__________________________________________________________________________________________________
token (InputLayer)              (None, None)         0                                            
__________________________________________________________________________________________________
time_distributed_1 (TimeDistrib (None, None, None, 3 114016      char[0][0]                       
__________________________________________________________________________________________________
word_embedding (Embedding)      (None, None, 64)     2539072     token[0][0]                      
__________________________________________________________________________________________________
char_lstm 

2018-11-18 23:43:50,848 - trainer.py[line:139] - INFO: train/valid set: 45932/11483
2018-11-18 23:43:50,849 - trainer.py[line:89] - INFO: use bucket sequence to speed up model training
2018-11-18 23:43:50,851 - sequence.py[line:346] - INFO: Training with 77 non-empty buckets
2018-11-18 23:43:51,206 - sequence.py[line:346] - INFO: Training with 77 non-empty buckets


mointor training process using f1 score and label acc
Successfully made a directory: models/word_rnn_201811182343
using Early Stopping
using Reduce LR On Plateau
tracking loss history and metrics


2018-11-18 23:43:52,132 - trainer.py[line:170] - INFO: saving model parameters and transformer to models/word_rnn_201811182343


model hyperparameters:
 {'nb_classes': 4, 'nb_tokens': 39673, 'maxlen': None, 'embedding_dim': 64, 'rnn_type': 'lstm', 'nb_rnn_layers': 2, 'drop_rate': 0.5, 're_drop_rate': 0.15, 'use_crf': True, 'inner_char': True, 'word_rnn_size': 128, 'integration_method': 'attention', 'char_feature_method': 'rnn', 'max_charlen': 10, 'nb_char_tokens': 3563, 'char_embedding_dim': 32, 'char_rnn_size': 32}
Epoch 1/25
535/753 [====================>.........] - ETA: 48s - loss: 0.4368 - acc: 0.8540

### 10-fold training

In [ ]:
config['train']['train_mode'] = 'fold'
seq_labeler_new = Labeler(model_name=model_name, transformer=transformer, seq_type='bucket', config=config)
seq_labeler_new.train(x,y)

## Predict New Samples

### Load data and transformer

In [ ]:
dataset = Dataset(fname='../data/cv_word_predict.txt',
                  task_type='sequence_labeling', mode='predict',
                  tran_fname='models/word_rnn_201811102046/transformer.h5')
x_seq = dataset.transform()

### Load model

In [ ]:
seq_labeler = Labeler('word_rnn', dataset.transformer)
seq_labeler.load(weight_fname='models/word_rnn_201811102046/model_weights_02_0.9273_0.8938.h5',
                 para_fname='models/word_rnn_201811102046/model_parameters.json')

### predict samples

In [ ]:
y_pred = seq_labeler.predict(x_seq)

In [ ]:
x_len = x_seq['length']
y_pred_true = [y_pred[i][:x_len[i]] for i in range(len(x_len))]
print([(x, y) for x, y in zip(dataset.texts[0].split(' '), y_pred_true[0])])